In [10]:
# Loading the files to understand its structure
import pandas as pd

file_path_1 = "/content/Players.csv"
df1 = pd.read_csv(file_path_1)

file_path_2 = "/content/Goalkeepers.csv"
df2 = pd.read_csv(file_path_2)


df1.rename(columns={df1.columns[-2]: 'Team', df1.columns[-3]: 'Year'}, inplace=True)
df2.rename(columns={df2.columns[-1]: 'Team', df2.columns[-2]: 'Year'}, inplace=True)

In [12]:
# Saving the updated Players.csv file
players_path = '/content/Players.csv'
df1.to_csv(players_path, index=False)

# Saving the updated Goalkeepers.csv file
goalkeepers_path = '/content/Goalkeepers.csv'
df2.to_csv(goalkeepers_path, index=False)

# Confirming that the files have been saved
players_path, goalkeepers_path


('/content/Players.csv', '/content/Goalkeepers.csv')

In [17]:
df1_path = "/content/Players.csv"


df2_path = "/content/Goalkeepers.csv"


df1 = pd.read_csv(df1_path, usecols=["Player", "Pos", "Age", "MP", "Gls", "Ast", "Year", "Team"])

df2 = pd.read_csv(df2_path, usecols=["Player", "Pos", "Age", "MP", "GA", "Saves", "Save%", "Year", "Team"])

,Player,Pos,Age,MP,GA,Saves,Save%,Year,Team
0,Ederson,GK,28.0,35,32.0,46.0,62.3,2022,Manchester City
1,Stefan Ortega,GK,29.0,3,1.0,14.0,93.3,2022,Manchester City
2,Aaron Ramsdale,GK,24.0,38,43.0,94.0,70.6,2022,Arsenal
3,David de Gea,GK,31.0,38,43.0,99.0,71.1,2022,Manchester United
4,Nick Pope,GK,30.0,37,32.0,85.0,74.6,2022,Newcastle United


In [20]:
# Defining the weights and adjustments
w_Gls = 3
w_Ast = 2
w_MP = 1
w_Age = 0.5
age_below_20_adjustment = 10
age_above_30_adjustment = 5

# Calculating the Experience Factor (EF)
df1['EF'] = w_MP * df1['MP'] - w_Age * (df1['Age'] - 25).abs()

# Applying Prime Age Adjustment
df1['Prime_Age_Adjustment'] = 0
df1.loc[df1['Age'] < 20, 'Prime_Age_Adjustment'] = age_below_20_adjustment
df1.loc[df1['Age'] > 30, 'Prime_Age_Adjustment'] = age_above_30_adjustment

# Calculating the overall score
df1['Score'] = (w_Gls * df1['Gls'] +
                w_Ast * df1['Ast'] +
                df1['EF'] -
                df1['Prime_Age_Adjustment'])

# Dropping temporary columns used for calculation
df1.drop(['EF', 'Prime_Age_Adjustment'], axis=1, inplace=True)


In [22]:
# Defining the weights
w1, w2 = 0.6, 0.4
w3, w4 = 0.1, 0.6

# Defining the save_score
df2['save_score'] = (df2['Saves'] * w1) + (df2['Save%'] * w2)

# Calculating the final_score
df2['score'] = df2['save_score'] + (df2['Age'] * w3) - (df2['GA'] * w4)

df2.drop(['save_score'], axis=1, inplace=True)

In [24]:
# Saving the updated Players.csv file
players_path = '/content/Players.csv'
df1.to_csv(players_path, index=False)

# Saving the updated Goalkeepers.csv file
goalkeepers_path = '/content/Goalkeepers.csv'
df2.to_csv(goalkeepers_path, index=False)

# Confirming that the files have been saved
players_path, goalkeepers_path

('/content/Players.csv', '/content/Goalkeepers.csv')

In [28]:
df1_path = "/content/Players.csv"


df2_path = "/content/Goalkeepers.csv"

df1 = pd.read_csv(df1_path)
df2 = pd.read_csv(df2_path)

In [31]:
# Merge the dataframes on the basis of Player name, Year, Team
merged_df = df1.merge(df2[['Player', 'Year', 'Team', 'score']], on=['Player', 'Year', 'Team'], how='left')

# Fill any NaN values in the score column with 0 (for players that are not in df2)
merged_df['score'].fillna(0, inplace=True)

# Add the score of df2 to the Score of df1
merged_df['Score'] += merged_df['score']

# Drop the temporary 'score' column
merged_df.drop(columns='score', inplace=True)

# Update df1 with the merged data
df1 = merged_df

# Show the first few rows of the updated df1
df1.head()


,Player,Pos,Age,MP,Gls,Ast,Year,Team,Score
0,Ederson,GK,28.0,35.0,0.0,1.0,2022,Manchester City,71.62
1,Rodri,MF,26.0,36.0,2.0,6.0,2022,Manchester City,53.50
2,Erling Haaland,FW,22.0,35.0,36.0,8.0,2022,Manchester City,157.50
3,Kevin De Bruyne,MF,31.0,32.0,7.0,16.0,2022,Manchester City,77.00
4,İlkay Gündoğan,MF,31.0,31.0,8.0,4.0,2022,Manchester City,55.00


In [35]:
df1['Score'].fillna(0, inplace=True)
nan_in_score = df1['Score'].isnull().sum()
nan_in_score

0

In [36]:
# Group by 'Player' and calculate the mean 'Score' for each player
average_scores = df1.groupby('Player')['Score'].mean().reset_index()
average_scores.rename(columns={'Score': 'Fscore'}, inplace=True)

# Merge the average scores back into the original dataframe
df1 = df1.merge(average_scores, on='Player', how='left')

# Show the first few rows of the updated df1 with the new 'Fscore' column
df1.head()


,Player,Pos,Age,MP,Gls,Ast,Year,Team,Score,Fscore
0,Ederson,GK,28.0,35.0,0.0,1.0,2022,Manchester City,71.62,85.28
1,Rodri,MF,26.0,36.0,2.0,6.0,2022,Manchester City,53.50,50.50
2,Erling Haaland,FW,22.0,35.0,36.0,8.0,2022,Manchester City,157.50,157.50
3,Kevin De Bruyne,MF,31.0,32.0,7.0,16.0,2022,Manchester City,77.00,85.75
4,İlkay Gündoğan,MF,31.0,31.0,8.0,4.0,2022,Manchester City,55.00,54.50


In [37]:
players_path = '/content/Players1.csv'
df1.to_csv(players_path, index=False)

In [38]:
# Group by 'Year' and 'Team' and calculate the mean 'Fscore' for each combination
df3 = df1.groupby(['Year', 'Team'])['Fscore'].mean().reset_index()

# Rename the 'Fscore' column to 'Score' in df3
df3.rename(columns={'Fscore': 'Score'}, inplace=True)

# Show the first few rows of df3
df3.head(30)


,Year,Team,Score
0,2019,Arsenal,20.859657
1,2019,Aston Villa,17.458333
2,2019,Bournemouth,11.641129
3,2019,Brighton and Hove Albion,12.226207
4,2019,Burnley,13.106869


In [40]:
file_path = '/content/premier_league.csv'
data = pd.read_csv(file_path)

In [41]:
# Convert the 'Season' column in premier_league_df to integer type to match the 'Year' column in df3
data['Season'] = data['Season'].astype(int)

# Merge premier_league_df with df3 on the basis of 'Season' (Year) and 'Team', adding the 'Score' from df3
merged_data = data.merge(df3, left_on=['Season', 'Team'], right_on=['Year', 'Team'], how='left')

# Drop the temporary 'Year' column
merged_data.drop(columns='Year', inplace=True)

# Show the first few rows of the merged dataframe
merged_data.head()


,Unnamed: 0,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,...,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Season,Team,Score
0,0,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,...,75.0,62443.0,İlkay Gündoğan,4-3-3,Michael Oliver,Match Report,NaN,2022,Manchester City,30.348177
1,1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,...,67.0,53453.0,İlkay Gündoğan,4-2-3-1,David Coote,Match Report,NaN,2022,Manchester City,30.348177
2,2,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,...,69.0,52258.0,İlkay Gündoğan,4-3-3,Jarred Gillett,Match Report,NaN,2022,Manchester City,30.348177
3,3,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,...,74.0,53112.0,Kevin De Bruyne,4-2-3-1,Darren England,Match Report,NaN,2022,Manchester City,30.348177
4,4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,...,74.0,53409.0,İlkay Gündoğan,4-2-3-1,Paul Tierney,Match Report,NaN,2022,Manchester City,30.348177


In [42]:
path = '/content/Game.csv'
merged_data.to_csv(path, index=False)